In [ ]:
import pandas as pd

df = pd.read_csv('data/king_country_houses.csv')

print(df.head())
print(df.info())
print(df.describe())
print(df.isnull().sum())


In [ ]:
import pandas as pd

# Load DF:
df = pd.read_csv(r'data\king_country_houses.csv')

# Handle missing values:
df = df.dropna(subset=['price'])

# Remove duplicate rows:
df = df.drop_duplicates()

# Convert 'date' column to datetime:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%dT%H%M%S')

# Check data:
print(df.info())


In [ ]:
# Calculate correlation between numerical features:
correlation_matrix = df.corr()
print(correlation_matrix['price'].sort_values(ascending=False))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Filter houses by price (650k):
df_filtered = df[df['price'] >= 650000]

# Correlation matrix:
correlation_matrix = df_filtered.corr()

# Visualize correlation matrix with heatmap:
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix[['price']].sort_values(by='price', ascending=False), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation of Features with Price')
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


# 1. Boxplot & Histogram combined in a grid:
def visualize_outliers(df):
    num_features = df.select_dtypes(include=['float64', 'int64']).columns
    num_plots = len(num_features)
    
    # Create subplots for Boxplots and Histograms:
    fig, axes = plt.subplots(num_plots, 2, figsize=(15, 5 * num_plots)) 
    
    for i, feature in enumerate(num_features):
        sns.boxplot(x=df[feature], ax=axes[i, 0])
        axes[i, 0].set_title(f'Boxplot for {feature}')
        
        df[feature].hist(bins=30, edgecolor='black', ax=axes[i, 1])
        axes[i, 1].set_title(f'Histogram for {feature}')
    
    plt.tight_layout()
    plt.show()

# 2. Z-score Method:
def z_score_outliers(df):
    num_features = df.select_dtypes(include=['float64', 'int64']).columns
    outliers = {}
    
    for feature in num_features:
        z_scores = stats.zscore(df[feature].dropna())
        outliers[feature] = np.where(np.abs(z_scores) > 3)[0]  # Indices of outliers
    
    return outliers

# 3. IQR Method:
def iqr_outliers(df):
    num_features = df.select_dtypes(include=['float64', 'int64']).columns
    outliers = {}
    
    for feature in num_features:
        Q1 = df[feature].quantile(0.25)
        Q3 = df[feature].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers[feature] = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)].index.tolist()
    
    return outliers

# Apply functions:
visualize_outliers(df)  # Visualize boxplots and histograms.
z_outliers = z_score_outliers(df)  # Detect outliers using Z-score.
iqr_outliers = iqr_outliers(df)  # Detect outliers using IQR.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Check for non-numeric columns:
non_numeric_columns = df.select_dtypes(exclude=[np.number]).columns

# Drop non-numeric columns:
df_cleaned = df.drop(columns=non_numeric_columns)

# Capping outliers using 5th and 95th percentiles:
numeric_columns = df_cleaned.select_dtypes(include=[np.number]).columns

# Calculate 5th and 95th percentiles for each numeric column:
lower_limit = df_cleaned[numeric_columns].quantile(0.05)
upper_limit = df_cleaned[numeric_columns].quantile(0.95)

# Apply capping for each column:
for col in numeric_columns:
    df_cleaned[col] = df_cleaned[col].clip(lower=lower_limit[col], upper=upper_limit[col])

# Filter relevant columns for model:
X = df_cleaned.drop("price", axis=1)
y = df_cleaned["price"]

# Split data into training and testing datasets:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize & train Random Forest Regressor Model with parallelization:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)  # n_jobs=-1 to use all CPU cores
rf_model.fit(X_train, y_train)

# Make predictions on the test dataset:
predictions = rf_model.predict(X_test)

# Evaluate model's performance:
r2 = r2_score(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
mae = mean_absolute_error(y_test, predictions)
mse_value = mean_squared_error(y_test, predictions)

# Evaluation metrics:
print("R² = ", round(r2, 4))
print("RMSE = ", round(rmse, 4))
print("MAE = ", round(mae, 4))
print("MSE =  ", round(mse_value, 4))


In [ ]:
import xgboost as xgb
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Generate synthetic regression data:
X, y = make_regression(n_samples=10000, n_features=5, noise=0.1, random_state=42)

# Apply outlier capping to target variable (y):
lower_percentile = 1
upper_percentile = 99

# Calculate lower and upper bounds for capping:
lower_bound = np.percentile(y, lower_percentile)
upper_bound = np.percentile(y, upper_percentile)

# Cap target variable within bounds:
y_capped = np.clip(y, lower_bound, upper_bound)

# Split into train and test sets:
X_train, X_test, y_train, y_test = train_test_split(X, y_capped, test_size=0.2, random_state=42)

# Convert data to DMatrix:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Train model:
bst = xgb.train({'objective': 'reg:squarederror'}, dtrain, num_boost_round=100)

# Make predictions:
predictions = bst.predict(dtest)

# Evaluate model:
r2 = r2_score(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
mae = mean_absolute_error(y_test, predictions)
mse_value = mean_squared_error(y_test, predictions)

print("R² = ", round(r2, 4))
print("RMSE = ", round(rmse, 4))
print("MAE = ", round(mae, 4))
print("MSE =  ", round(mse_value, 4))


In [ ]:
from sklearn.metrics import make_scorer, mean_squared_error, root_mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

# Define model:
model = XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=6)

# Define scorer for RMSE:
scorer = make_scorer(root_mean_squared_error)

# Perform cross-validation:
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring=scorer)

# Print scores:
print(f"Cross-Validation RMSE Scores: {cv_scores}")
print(f"Mean Cross-Validation RMSE: {cv_scores.mean()}")
